In [1]:
import os
import gc
from collections import OrderedDict
import snntorch as snn

from typing import Tuple, List, Dict

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import numpy as np

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [23]:
def gpu_mem_state():
  # Print out the GPU memory usage
  print("Memory allocated:", torch.cuda.memory_allocated() / 1024**3, "GB")
  print("Max memory allocated:", torch.cuda.max_memory_allocated() / 1024**3, "GB")

gpu_mem_state()

Memory allocated: 0.0 GB
Max memory allocated: 0.0 GB


In [2]:
from pathlib import Path


class ResonatorSpikes:

    def __init__(self, clk_freq, resonator_freq, spikes_path):
        self.clk_freq = clk_freq
        self.resonator_freq = resonator_freq
        self.events = None
        self._load_spikes(spikes_path)

    def _load_spikes(self, spikes_path):
        spikes_array = np.load(spikes_path)['spikes']
        # if the file is already events based spikes
        if np.max(spikes_array) > 1:
            self.events = spikes_array
        else:
            self.events = np.where(spikes_array == 1)[0]

    def spectrogram(self, window_ms):
        window = int(self.clk_freq/1000 * window_ms)
        N = self.events[-1] // window + 1
        bins = np.zeros(N, dtype=int)
        unique_indices, counts = np.unique(np.array(self.events) // window, return_counts=True)
        bins[unique_indices] = counts
        return bins


class ChannelSpikes:

    def __init__(self, base_folder, channel_name):
        self.channel_name = channel_name
        self.resonators_output = OrderedDict({})
        self._load_resonators_output(base_folder)

    def _load_resonators_output(self, base_folder):
        channel_folder = base_folder / self.channel_name
        for clk_freq in os.listdir(channel_folder):
            clk_folder = channel_folder / clk_freq
            for spikes in os.listdir(clk_folder):
                resonator_freq = spikes[:-4]
                self.resonators_output[resonator_freq] = ResonatorSpikes(int(clk_freq), float(resonator_freq), f'{clk_folder}/{spikes}')

class SignalSpikes:

    def __init__(self, signal_folder, label):
        self.label = label
        self.channels = OrderedDict({
            channel: ChannelSpikes(signal_folder, channel)
            for channel in os.listdir(signal_folder)
        })


class Trial:

    def __init__(self, base_folder, trial):
        self.trial = trial
        self.base_folder = Path(f'{base_folder}/{trial}')

    def load(self, minute):
        # make sure it's in string format.
        minute = str(minute)
        for label in os.listdir(self.base_folder):
            for m in os.listdir(self.base_folder / label):
                if m == minute:
                    return SignalSpikes(self.base_folder / label / m, label=label)

In [103]:


class EEGMentalSpikesDataset(Dataset):
    def __init__(self, trials: List[Trial], minutes: List[int], time_sample_s: float, labels_mapper: Dict[str, int]):
        self.time_sample_s = time_sample_s
        self.labels_mapper = labels_mapper

        self.samples_per_minute = int(60 / time_sample_s)
        self.samples_per_trial = self.samples_per_minute * len(minutes)
        self.length = len(trials) * self.samples_per_trial

        self.loaded_spikes = {
            f'{i}-{j}': trial.load(minute)
            for i, trial in enumerate(trials)
            for j, minute in enumerate(minutes)
        }
        # get resonators clk frequencies.
        signal4example = next(iter(self.loaded_spikes.values()))
        resonators = next(iter(signal4example.channels.values())).resonators_output

        self.map_channel_to_id = {ch: i for i, ch in enumerate(signal4example.channels.keys())}
        self.map_resonator_to_id = {f: i for i, f in enumerate(resonators.keys())}

        clk_freq = list(set(map(lambda x: x.clk_freq, resonators.values())))
        # least common multiplier of the resonators is network clk frequency
        self.network_clk = np.lcm.reduce(clk_freq)

    def __len__(self):
        return self.length

    def __getitem__(self, id):
        num_rows = 40_000

        trial_id = id // self.samples_per_trial
        minute_id = (id % self.samples_per_trial) // self.samples_per_minute
        sample = ((id % self.samples_per_trial) % self.samples_per_minute)

        spike_signal = self.loaded_spikes[f'{trial_id}-{minute_id}']
        label = self.labels_mapper[spike_signal.label]


        # ordered dict to numpy array
        # Determine the size of the resulting array
        result = -np.ones((num_rows, 2), dtype=np.int64)
        result_index = 0

        for ch, channel_spikes in spike_signal.channels.items():
            ch_id = self.map_channel_to_id[ch]
            for f, resonator in channel_spikes.resonators_output.items():
                resonator_id = self.map_resonator_to_id[f]
                ticks_in_sample = int(self.time_sample_s * resonator.clk_freq)

                ts_spikes = resonator.events
                ts_spikes = ts_spikes[(ts_spikes >= (sample * ticks_in_sample)) & (ts_spikes < ((sample + 1) * ticks_in_sample))]

                # make sure all spikes are aligned even though the spikes come from different clocks and different timestamp!
                ts_spikes -= (sample * ticks_in_sample)
                ts_spikes = ts_spikes * int(self.network_clk // resonator.clk_freq)

                neuron_id = ch_id * len(self.map_resonator_to_id) + resonator_id
                result[result_index:result_index + len(ts_spikes), 0] = ts_spikes
                result[result_index:result_index + len(ts_spikes), 1] = neuron_id
                result_index += len(ts_spikes)

        return result, label

In [154]:
trial = Trial(f'../datasets/EEG_data_for_Mental_Attention_State_Detection/EEG_spikes_clk/', 3)

labels_mapper = {
    'drowesed': 0,
    'focus': 1,
    'unfocus': 2,
}
train_dataset = EEGMentalSpikesDataset(trials=[trial], minutes=[5, 15, 25], time_sample_s=.05, labels_mapper=labels_mapper)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

val_dataset = EEGMentalSpikesDataset(trials=[trial], minutes=[6, 16, 26], time_sample_s=.05, labels_mapper=labels_mapper)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)

In [155]:
import time

st = time.time()
x, targets = next(iter(train_dataloader))
time.time() - st

0.19999051094055176

In [106]:
x[:, :10, 0]

tensor([[ 372,  792, 1232, 1660, 2080, 2520, 2956, 3376, 3816, 4236],
        [ 144,  828, 1600, 2040, 2476, 2912, 3352, 3772, 4200, 4640],
        [ 324,  744, 1184, 1604, 2024, 2464, 2884, 3304, 3744, 4180],
        [   4,  432,  872, 1292, 1728, 2168, 2588, 3008, 3428, 3868],
        [ 368,  788, 1296, 1736, 2164, 2944, 3380, 3820, 4240, 4680],
        [ 104,  540,  980, 1400, 1840, 2260, 2680, 3120, 3540, 3960],
        [ 316,  756, 1184, 1612, 2052, 2472, 2912, 3332, 3752, 4192],
        [ 252,  672, 1092, 1532, 1952, 2372, 2812, 3232, 3652, 4092],
        [ 412,  832, 1252, 1692, 1848, 2288, 2708, 3128, 3568, 3988],
        [ 168,  588, 1024, 1464, 1884, 2304, 2724, 3164, 3592, 4012],
        [ 192,  788, 1576, 1996, 2436, 2856, 3276, 3716, 4136, 4556],
        [ 376,  816, 1236, 1672, 2112, 2532, 2972, 3392, 3812, 4252],
        [   8,  448,  868, 1304, 1740, 2180, 2600, 3020, 3440, 3880],
        [ 216,  656, 1076, 1496, 1936, 2356, 2792, 3212, 3652, 4088],
        [ 200,  640,

In [111]:
y = np.zeros((x.shape[0], 25*14))
indices = (x[:, :, 0] == 329).nonzero()
features = x[indices[:, 0], indices[:, 1], 1]
y[indices[:, 0], features] = 1

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [123]:
x[4, x[4, :, 0] == 372, 1]

tensor([  5, 194, 212])

## Define The Network.

In [186]:
# Define Network
class SNN(nn.Module):
    def __init__(self, network_clk, sample_time_s, num_inputs, beta=.95):
        super().__init__()

        self.network_clk = network_clk
        self.sample_time_s = sample_time_s
        self.steps = 5000 #int(self.network_clk * self.sample_time_s)

        self.fc1 = nn.Linear(num_inputs, num_inputs * 3)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_inputs * 3, len(labels_mapper))
        self.lif2 = snn.Leaky(beta=beta)

    def event_ts_to_spikes(self, events_ts, t):
        x = np.zeros((events_ts.shape[0], 25*14))
        indices = (events_ts[:, :, 0] == t).nonzero()
        features = events_ts[indices[:, 0], indices[:, 1], 1]
        x[indices[:, 0], features] = 1
        return torch.tensor(x, requires_grad=True).float()

    def forward(self, events_ts):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif1.init_leaky()
        
        # Record the final layer
        spk_rec = []
        mem_rec = []
        for i in range(self.steps):
            spikes = self.event_ts_to_spikes(events_ts, i)
            spikes = spikes.to(device)
            cur1 = self.fc1(spikes)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)

            spk_rec.append(spk2)
            mem_rec.append(mem2)

        return torch.stack(spk_rec, dim=0), torch.stack(mem_rec, dim=0)
        
# Load the network onto CUDA if available
net = SNN(train_dataset.network_clk, sample_time_s=.05, num_inputs=14*25).to(device)

gpu_mem_state()

Memory allocated: 0.0 GB
Max memory allocated: 0.0 GB


In [187]:
def train_printer(
    data, targets, epoch,
    counter, iter_counter,
        loss_hist, test_loss_hist, test_data, test_targets):
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print("\n")

loss = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

gpu_mem_state()

Memory allocated: 0.0 GB
Max memory allocated: 0.0 GB


### Training Loop

In [188]:
import torch.nn.functional as F


num_epochs = 1
loss_hist = []
val_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_dataloader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)
        # targets = F.one_hot(targets, num_classes=len(labels_mapper))

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data)

        _, idx = spk_rec.sum(dim=0).max(1)

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(net.steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        train_acc = np.mean((targets == idx).detach().cpu().numpy())
        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Val set
        with torch.no_grad():
            net.eval()
            val_data, val_targets = next(iter(val_dataloader))
            val_data = val_data.to(device)
            val_targets = val_targets.to(device)

            # Val set forward pass
            val_spk, val_mem = net(val_data)

            _, idx = val_spk.sum(dim=0).max(1)
            val_acc = np.mean((val_targets == idx).detach().cpu().numpy())

            # Val set loss
            val_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(net.steps):
                val_loss += loss(val_mem[step], val_targets)
            val_loss_hist.append(val_loss.item())

            # Print train/val loss/accuracy
            if counter % 1 == 0:
              print(f'train acc {train_acc}, val acc {val_acc}')
              train_printer(
                  data, targets, epoch,
                  counter, iter_counter,
                  loss_hist, val_loss_hist,
                  val_data, val_targets)
            counter += 1
            iter_counter +=1

train acc 0.5, val acc 0.5
Epoch 0, Iteration 0
Train Set Loss: 7616.81
Test Set Loss: 5032.32


train acc 0.0, val acc 1.0
Epoch 0, Iteration 1
Train Set Loss: 5414.84
Test Set Loss: 6779.80




KeyboardInterrupt: 

## ANN

In [ ]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(14, 32, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(3, 3))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 3))
        self.conv3 = nn.Conv2d(64, 64, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        self.fc1 = nn.Linear(179456, 1024)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(1024, 3)
        
    def forward(self, x):
        # print(x.dtype)
        x = self.conv1(x.float())
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x




# Load the network onto CUDA if available
cnet = CNN().to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnet.parameters(), lr=5e-4, betas=(0.9, 0.999))

gpu_mem_state()

Memory allocated: 0.6859326362609863 GB
Max memory allocated: 0.6859326362609863 GB


In [ ]:
# del data
# del targets
gc.collect()
torch.cuda.empty_cache()
gpu_mem_state()

Memory allocated: 0.6859326362609863 GB
Max memory allocated: 0.6859326362609863 GB


In [ ]:
from tqdm import tqdm
# Define the train and validation loops
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    train_correct = 0
    for data, target in tqdm(train_loader, desc="Training", leave=False):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    return train_loss, train_acc

def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for data, target in  tqdm(val_loader, desc="Validation", leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            val_correct += pred.eq(target.view_as(pred)).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc = 100. * val_correct / len(val_loader.dataset)
    return val_loss, val_acc

# Train and validate the CNN
n_epochs = 10
for epoch in range(1, n_epochs + 1):
    train_loss, train_acc = train(cnet, train_loader, optimizer, loss, device)
    val_loss, val_acc = validate(cnet, val_loader, loss, device)
    print(f'Epoch {epoch}: Train Loss: {train_loss:.6f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.6f}, Val Acc: {val_acc:.2f}%')

Epoch 1: Train Loss: 0.047621, Train Acc: 32.21%, Val Loss: 0.046404, Val Acc: 34.48%


Epoch 2: Train Loss: 0.045774, Train Acc: 34.48%, Val Loss: 0.046393, Val Acc: 34.48%


Epoch 3: Train Loss: 0.045769, Train Acc: 34.48%, Val Loss: 0.046397, Val Acc: 34.48%


Epoch 4: Train Loss: 0.045768, Train Acc: 34.48%, Val Loss: 0.046395, Val Acc: 34.48%


Epoch 5: Train Loss: 0.045768, Train Acc: 34.48%, Val Loss: 0.046392, Val Acc: 34.48%


Epoch 6: Train Loss: 0.045768, Train Acc: 34.48%, Val Loss: 0.046390, Val Acc: 34.48%


KeyboardInterrupt: ignored

In [ ]:
torch.save(cnet.state_dict(), '"/content/drive/MyDrive/SNN-Thesis/cnn_trained_model.pth')

In [ ]:
import requests

api_key = '8236F572-BB36-431D-A64C-3A21B2751024'

symbol = 'BTC_USDT'.upper().replace('-', '_')
trades = []
endpoint = f'https://rest.coinapi.io/v1/trades/BINANCEFTS_PERP_{symbol}/history'
params = {
    'apikey': api_key,
    'time_start': '2023-03-21T08:59:54+00:00',
    'limit': 100,
}
response = requests.get(endpoint, params=params, headers={'Accept': 'application/json'})


In [ ]:
response

<Response [500]>